In [1]:
import os
import json
from tqdm import trange
import numpy as np

from tqdm import tqdm
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
path_dir = 'D:/사람동작 영상/annotation/2D'
file_list1 = os.listdir(path_dir)

In [ ]:
false_data = list()
true_data = list()

true_name = list()

for file_name1 in tqdm(file_list1):
    #print(file_name1)
    file_path = path_dir + '/' + file_name1

    file_list2 = os.listdir(file_path)

    if file_name1[0] == '6': # 쓰러짐 영상
        for file_name2 in file_list2:
            read_file_path = file_path + '/' + file_name2
            true_name.append(file_name2)

            with open(read_file_path, 'r') as f:
                json_data = json.load(f)
                #print(json_data.keys())
                tmpList = list()

                for data in json_data['annotations']:
                    tmpList.append(data['keypoints'])

                true_data.append(tmpList)

    else: # 기타 영상
        for file_name2 in file_list2:
            read_file_path = file_path + '/' + file_name2


            with open(read_file_path, 'r') as f:
                json_data = json.load(f)
                #print(json_data.keys())
                tmpList = list()
                for data in json_data['annotations']:
                    tmpList.append(data['keypoints'])

                false_data.append(tmpList)

 59%|█████▉    | 206/348 [05:03<02:33,  1.08s/it]

In [7]:
num_list = list()
for data in true_data:
    num_list.append(len(data))

print(np.mean(num_list))

21.000522056904202


In [13]:
for index, data in enumerate(true_data):
    if len(data) < 30:
        tmp_list = [[0 for w in range(48)] for h in range(30-len(data))]
        data += tmp_list
        true_data[index]=data
    else:
        true_data[index] = data[:30]

In [14]:
for index, data in enumerate(false_data):
    if len(data) < 30:
        tmp_list = [[0 for w in range(48)] for h in range(30-len(data))]
        data += tmp_list
        false_data[index]=data
    else:
        false_data[index] = data[:30]

In [15]:
true_np = np.array(true_data, dtype=np.float32)
false_np = np.array(false_data, dtype=np.float32)

In [16]:
print(false_np.shape)
print(true_np.shape)

(198335, 30, 48)
(3831, 30, 48)


In [17]:
index = [(3*i)-1 for i in range(1, 17)]

In [18]:
false_np = np.delete(false_np, index, 2)
true_np = np.delete(true_np, index, 2)

In [19]:
import torch

In [22]:
true_np = np.repeat(true_np, 30, axis=0)

In [23]:
print(true_np.shape)

(114930, 30, 32)


In [26]:
y_true = np.array([1 for w in range(len(true_np))])
y_false = np.array([0 for w in range(len(false_np))])

In [27]:
data_x = np.concatenate((false_np, true_np), axis=0)
data_y = np.concatenate((y_false, y_true), axis=0)

In [28]:
print(data_x.shape)
print(data_y.shape)

(313265, 30, 32)
(313265,)


In [29]:
import torch
from torch.utils.data import TensorDataset, DataLoader

In [30]:
idx = np.arange(data_x.shape[0])

In [31]:
print(idx)

[     0      1      2 ... 313262 313263 313264]


In [32]:
np.random.shuffle(idx)

In [33]:
print(idx)

[255851 138507 285039 ... 106103 106052  73519]


In [34]:
data_x = data_x[idx]
data_y = data_y[idx]

In [72]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)
split_idx = int(len(data_x)*0.8)
train_x, remaining_x = data_x[:split_idx], data_x[split_idx:]
train_y, remaining_y = data_y[:split_idx], data_y[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

In [78]:
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

In [80]:
torch.save(train_data, 'train_mul.pt')
torch.save(test_data, 'test_mul.pt')
torch.save(valid_data, 'valid_mul.pt')

In [83]:
len(true_data[0])

30

In [82]:
len(false_data)

198335

In [ ]:
print(true_name)

In [44]:
for arr in true_data:
    np_arr = np.array(arr, dtype=np.float32)
    
    np_arr = np.delete(np_arr, index, 1)
    
    print((data_x == np_arr).shape)

(313265, 30, 32)
(313265, 30, 32)
(313265, 30, 32)
(313265, 30, 32)
(313265, 30, 32)
(313265, 30, 32)
(313265, 30, 32)
(313265, 30, 32)


KeyboardInterrupt: 

In [66]:
np_list = [[0 for i in range(20)] for j in range (30)]

In [95]:
sub_mat = np.zeros((32, 20), dtype=int)
index_a = [9, 8, 8, 14, 11, 8, 6, 6, 4, 1]
index_b = [8, 14, 11, 15, 10, 6, 4, 1, 5, 0]

for i, (a, b) in enumerate(zip(index_a, index_b)):
    sub_mat[a*2, i*2] = 1
    sub_mat[b*2, i*2] = -1

    sub_mat[a*2+1, i*2 + 1] = 1
    sub_mat[b*2+1, i*2 + 1] = -1


print(sub_mat)

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  1  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 -1  0  1  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 -1  0  1  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0

In [116]:
num_list = list()

for data in tqdm(data_x):
    
    np_arr = data@sub_mat
    
    #if(np.ptp(np_arr) != 0):
    #    np_arr = (np_arr - np.min(np_arr))/np.ptp(np_arr)
        
    #print(np_arr.shape)
    
    num_list.append(np_arr)

100%|██████████| 313265/313265 [00:02<00:00, 120336.34it/s]


In [117]:
new_np = np.array(num_list, dtype=np.float32)
new_np.shape

(313265, 30, 20)

In [118]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)
split_idx = int(len(data_x)*0.8)
train_x, remaining_x = new_np[:split_idx], new_np[split_idx:]
train_y, remaining_y = data_y[:split_idx], data_y[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

torch.save(train_data, 'train_v2.pt')
torch.save(test_data, 'test_v2.pt')
torch.save(valid_data, 'valid_v2.pt')

In [114]:
print(train_x.shape)
print(train_y.shape)

(250612, 30, 20)
(250612,)


In [115]:
print(new_np[0])

[[0.6158192  0.36158192 1.         0.40112993 0.47740114 0.15536723
  0.33050847 0.47457626 0.5423729  0.7627119  0.5706215  0.
  0.48022598 0.45480227 0.69491524 0.2881356  0.5536723  0.37853107
  0.84463274 0.31920904]
 [0.700565   0.41525424 0.87288135 0.36158192 0.7259887  0.22881356
  0.46892655 0.4180791  0.519774   0.43502825 0.81920904 0.12146892
  0.6129944  0.37288135 0.28248587 0.47740114 0.75706214 0.34463277
  0.5423729  0.3983051 ]
 [0.759887   0.4915254  0.8163842  0.44915253 0.43502825 0.3022599
  0.4463277  0.46327683 0.4858757  0.45762712 0.88135594 0.26836157
  0.46892655 0.460452   0.25706214 0.5084746  0.7033898  0.4180791
  0.75706214 0.40112993]
 [0.69774014 0.5423729  0.7824859  0.48022598 0.52259886 0.36723164
  0.49717513 0.5141243  0.51129943 0.50282484 0.9463277  0.41525424
  0.5254237  0.45762712 0.38418078 0.53672314 0.7627119  0.5508475
  0.720339   0.4435028 ]
 [0.6779661  0.5875706  0.7655367  0.5        0.55649716 0.39265537
  0.50282484 0.5084746  0.5